# Transforming a time series of GeoTIFFs to a multidimensional CF compliant netCDF4 - GDAL and NCO example

---
## Background:  
---

This demonstration is covered in the NASA Earthdata Webinar, 
 [netCDF Why and How – Creating publication-quality netCDF datasets](https://daac.ornl.gov/resources/learning/)

For these examples, a dataset was originally provided to the ORNL DAAC as thousands of geotiff files representing SnowModel output for years 2005 - 2008.  Individual data files were also at 5 different spatial resolutions; 25-m, 100-m, 500-m, 2000-m, and 10000-m.  The individual geoTIFF files each contained 2-bands of data, SnowDepth and SnowDensity. The original dataset package consisted of over 5,400 individual files.

The final distribution data package from the ORNL DAAC transformed these geoTIFF files into CF Compliant netCDF file formats and arranged them into 1 file per year with a daily time step, separate SnowDepth and SnowDensity variables, and separate spatial resolutions.  Through this arrangement, the final dataset distribution package consisted of 40 netCDF files.

The original geoTIFF files for year 2006 are ONLY available with this demonstration and are only intended as demonstration data.  Not for scientific use.

This example references the dataset published by the [ORNL DAAC](https://daac.ornl.gov/)
- [ABoVE: Dall Sheep Response to Snow and Landscape Covariates, Alaska, 2005-2008](https://doi.org/10.3334/ORNLDAAC/1602).

## 1. Demonstration Data: 
---
This demonstration will use only the original 365 geoTIFF files for 2006.  Note, the original geoTIFF files contained 2-bands; SnowDepth and SnowDensity.  SnowDepth will be the only variable extracted for this demonstation.

Here is a sample of the original data, 365 files, for the year 2006.

    SnowModel_DALL_2006_01_01.tif   SnowModel_DALL_2006_01_02.tif
    SnowModel_DALL_2006_01_03.tif   SnowModel_DALL_2006_01_04.tif
    SnowModel_DALL_2006_01_05.tif   SnowModel_DALL_2006_01_06.tif
    SnowModel_DALL_2006_01_07.tif   SnowModel_DALL_2006_01_08.tif
    SnowModel_DALL_2006_01_09.tif   SnowModel_DALL_2006_01_10.tif
    SnowModel_DALL_2006_01_11.tif   SnowModel_DALL_2006_01_12.tif
     .....
    SnowModel_DALL_2006_12_30.tif   SnowModel_DALL_2006_12_31.tif
    
<img src="images/TimeSeriesSnowDepth_horizontal_crop.png" />

## 2. Software 
---

### Command Line Utilities

This demonstration uses a combination of the command line utilties, GDAL and NCO.

### 2.1 GDAL

[GDAL](https://gdal.org/) is a translation library for raster and vector geospatial data formats.

* [gdalinfo](https://gdal.org/programs/gdalinfo.html) - Lists information about a raster dataset.
* [gdal_translate](https://gdal.org/programs/gdal_translate.html) - Converts raster data between different formats.

[GDAL Conda Install](https://anaconda.org/conda-forge/gdal)

Example usage:
 
- __List information about a raster dataset:__

```cmd
gdalinfo infile.tif
```

- __Extract the first band from a multi-band raster file:__

```cmd
gdal_translate -b 1 input.tif output_band1.tif
```

- __Convert from a geoTIFF format to a netCDF format:__

```cmd
gdal_translate -of netCDF -co WRITE_BOTTOMUP=YES -co WRITE_LONLAT=YES in.tif out.nc
```

### 2.2 ncdump
[ncdump](https://www.unidata.ucar.edu/software/netcdf/netcdf/ncdump.html)
 A command line utility that generates a text representation of a specified netCDF file.
 
 Available with the [NCO conda install](https://anaconda.org/conda-forge/nco)

Example usage:
 
- __List the header of a netCDF file:__

```cmd
ncdump -h in.nc
```

- __List the header and the specified variable values of a netCDF file:__

```cmd
ncdump -v varname in.nc
```

### 2.3 NCO

### NCO
[NCO](http://nco.sourceforge.net/)  Comprise about a dozen standalone, command-line programs that operate on netCDF files.

[What is NCO](http://nco.sourceforge.net/#Definition).
The standalone, command-line operators are separated based on their general utility.  

Available with the [NCO conda install](https://anaconda.org/conda-forge/nco)

The 'geoTIFF to netCDF' demonstration makes use the following:
1. [ncatted](https://linux.die.net/man/1/ncatted) – netCDF ATTribute EDitor
2. [ncrename](https://linux.die.net/man/1/ncrename) – netCDF RENAMEer
3. [ncecat](https://linux.die.net/man/1/ncecat) – netCDF Ensemble conCATenator
4. [ncks](https://linux.die.net/man/1/ncks) – netCDF Kitchen Sink
5. [ncap2](https://linux.die.net/man/1/ncap2) - netCDF Arithmetic Processor
 
Example usage:
 
* __Rename a variable in a netCDF file:__

```cmd
ncrename -h -v old_name,new_name in.nc
```

- __Concatenate an arbitrary number of input flies into a single output file:__

```cmd
ncecat -h input_files output_file.nc
```

- __Add and populate the 'time' array variable and values:__

```cmd
ncap2 -s 'time=array(start, step, $time);' input_file.nc output_file.nc
```


- __Update and attribute to conform to CF standards:__

```cmd
ncatted attribute_name, variable_name, mode, att_type, att_val in.nc
```

> * mode = append, create, delete, modify, overwrite
> * att_type = float, double, long, short, char, byte

#### Examine a geoTIFF file with GDAL's gdalinfo:

In [1]:
gdalinfo Orig_SnowModel_DALL_2006_01_01.tif

Driver: GTiff/GeoTIFF
Files: Orig_SnowModel_DALL_2006_01_01.tif
Size is 160, 240
Coordinate System is:
PROJCS["unnamed",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.2572221010042,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Albers_Conic_Equal_Area"],
    PARAMETER["standard_parallel_1",55],
    PARAMETER["standard_parallel_2",65],
    PARAMETER["latitude_of_center",50],
    PARAMETER["longitude_of_center",-154],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]]]
Origin = (-28000.000000000000000,1230000.000000000000000)
Pixel Size = (500.000000000000000,-500.000000000000000)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLE

#### Examine a netCDF file with ncdump:

In [2]:
ncdump -h snow_depth_00500m_2006_time_final.nc

netcdf snow_depth_00500m_2006_time_final {
dimensions:
	time = UNLIMITED ; // (365 currently)
	x = 160 ;
	y = 240 ;
variables:
	double time(time) ;
		time:units = "days since 2006-01-01 00:00:00" ;
		time:calendar = "standard" ;
		time:description = "middle of each day" ;
		time:long_name = "time" ;
		time:standard_name = "time" ;
	double x(x) ;
		x:standard_name = "projection_x_coordinate" ;
		x:long_name = "x coordinate of projection" ;
		x:units = "m" ;
	double y(y) ;
		y:standard_name = "projection_y_coordinate" ;
		y:long_name = "y coordinate of projection" ;
		y:units = "m" ;
	float lat(y, x) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
	float lon(y, x) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
	float SnowDepth_500m(time, y, x) ;
		SnowDepth_500m:long_name = "SnowModel snow depth: 500-m resolution" ;
		SnowDepth_500m:_FillValue = -3.4e+38f ;
		SnowDepth_500m:grid_mapp

## 3. CF Compliance
---
The header of a netCDF file should follow standards as defined by the CF (Climate and Forecast) conventions.  
The conventions define metadata that provide a definitive description of what the data in each variable represents, and the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities.
* [CF Conventions and Metadata](http://cfconventions.org/)
* [Standard Name Table](http://cfconventions.org/Data/cf-standard-names/67/build/cf-standard-name-table.html)


***

---
# Demonstration 
---

## Convert a Series of GeoTIFF files to netCDF  

### Step 1:  Extract the SnowDepth Band from GeoTIFF files
---
#### The `gdal_translate` function of the GDAL operators can extract individual bands from a geoTIFF file

The -b flag selects the band of interest. In this case, Band1 is SnowDepth.

* To save processing time, for the webinar this is demonstrated below on just 1 file.  

In [3]:
# Step 1: Extract SnowDepth band from geoTIFF files
gdal_translate -b 1 SnowModel_DALL_2006_01_01.tif SnowModel_DALL_Band1_2006_01_01.tif

Input file size is 160, 240
0...10...20...30...40...50...60...70...80...90...100 - done.


#### Check that only Band 1 is in the new file

In [4]:
gdalinfo SnowModel_DALL_Band1_2006_01_01.tif

Driver: GTiff/GeoTIFF
Files: SnowModel_DALL_Band1_2006_01_01.tif
Size is 160, 240
Coordinate System is:
PROJCS["unnamed",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.2572221010042,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Albers_Conic_Equal_Area"],
    PARAMETER["standard_parallel_1",55],
    PARAMETER["standard_parallel_2",65],
    PARAMETER["latitude_of_center",50],
    PARAMETER["longitude_of_center",-154],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]]]
Origin = (-28000.000000000000000,1230000.000000000000000)
Pixel Size = (500.000000000000000,-500.000000000000000)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=BAND
Corner C

### Example scripts to loop through all files
For the full dataset, this step needs to be performed on all geoTIFF files. 
- Because these are command line utilities, use scripting to loop through and process all files.

Windows:
```cmd
for %f in (*.tif) do (gdal_translate -b 1 %f band1/%f)
```
UNIX:
```sh
for f in *.tif; do gdal_translate -b 1 $f band1/$f; done
```

### Step 2:  Convert the geoTIFF files to netCDF
---
#### 2.1 The `gdal_translate` function of the GDAL operators converts file formats, including geoTIFF to netCDF

The demonstration geoTIFF files are in a projected coordinate reference system, `Albers Conical Equal Area` with appropriate parameters.  CF Convensions require that when a coordinate variable for a grid are not latitude and longitude, the true latitude and longitude coordinates are supplied via the `coordinates` attribute. By using the gdal_translate creation option `"WRITE_LONLAT=YES"`, latitude and longitude will be added to the netCDF file as 2-dimensional lat and lon variables with values of each pixel centerpoint in decimal degrees.  

The translation will also provide a fully defined coordinate reference system in the `albers_conical_equal_area` variable and attributes and also assign the `grid_mapping` attribute to the data variables(s).

More information can be found at the following links:

[CF Conventions](http://cfconventions.org/)

[CF Conventions: Projections](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#grid-mappings-and-projections)

[Grid Mappings Appendix](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#appendix-grid-mappings)


In [5]:
# Step 2: Convert geoTIFF to netCDF
gdal_translate -of netCDF -co WRITE_LONLAT=YES SnowModel_DALL_Band1_2006_01_01.tif SnowModel_DALL_2006_01_01.nc

Input file size is 160, 240
0...10...20...30...40...50...60...70...80...90...100 - done.


#### 2.2 Examine the netCDF file with `ncdump`
*   Examine how the gdal_translate conversion from geoTIFF to netCDF captures most of the necessary netCDF header information
> * `x` and `y` dimension
> * `x` and `y` variables
> * 2D `lat` and `lon` variables; mapped to (y,x) dimensions
> * file projection information written to `albers_conical_equal_area` including `grid_mapping_name` and `grid_mapping` attributes in the projection and data `Band1` variable
> * data variable as `Band1` (SnowDepth)
> * Note that the files are only 2 dimensions; x and y .  No time dimension.

In [6]:
ncdump -h SnowModel_DALL_2006_01_01.nc
# The `-h` flag will print only the header to standout

netcdf SnowModel_DALL_2006_01_01 {
dimensions:
	x = 160 ;
	y = 240 ;
variables:
	char albers_conical_equal_area ;
		albers_conical_equal_area:grid_mapping_name = "albers_conical_equal_area" ;
		albers_conical_equal_area:false_easting = 0. ;
		albers_conical_equal_area:false_northing = 0. ;
		albers_conical_equal_area:latitude_of_projection_origin = 50. ;
		albers_conical_equal_area:longitude_of_central_meridian = -154. ;
		albers_conical_equal_area:standard_parallel = 55., 65. ;
		albers_conical_equal_area:long_name = "CRS definition" ;
		albers_conical_equal_area:longitude_of_prime_meridian = 0. ;
		albers_conical_equal_area:semi_major_axis = 6378137. ;
		albers_conical_equal_area:inverse_flattening = 298.257222101004 ;
		albers_conical_equal_area:spatial_ref = "PROJCS[\"unnamed\",GEOGCS[\"NAD83\",DATUM[\"North_American_Datum_1983\",SPHEROID[\"GRS 1980\",6378137,298.2572221010042,AUTHORITY[\"EPSG\",\"7019\"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY[\"EPSG\",\"6269\"]],PRIMEM[\"Greenwich\",0]

### Example scripts to loop through 365 individual *'Band 1' files for 2006; 1 file per day.

Windows
```cmd
for %f in (*.tif) do (gdal_translate -of netCDF -co WRITE_LONLAT=YES %f %~nf.nc)
```
UNIX:
```sh
for f in *.tif; do gdal_translate -of netCDF -co WRITE_LONLAT=YES $f ${f%.*}.nc; done   
```

#### After looping through all files, there are now 365 2-D netCDF files, created from Band1 of the 365 geoTIFF files for year 2006.

### Step 3. Use NCO's `ncrename` to rename the variable _albers-conical-equal-area_ to _crs_
---
**A couple extra steps are necessary to properly assign the `time` dimension to the SnowDepth variable.

Naming the projection system variable to `crs` is more standardized and allows `ncecat` (the netCDF operator that merges the 365 files into 1) to correctly assigns the time dimension
* the -h flag stops NCO operators from writing each command to the `// global attributes` section
* the -v flag tells ncrename to rename a `variable`

In [7]:
# Step 3: Rename albers_conical_equal_area variable to crs
ncrename -h -v albers_conical_equal_area,crs SnowModel_DALL_2006_01_01.nc

In [8]:
ncdump -h SnowModel_DALL_2006_01_01.nc

netcdf SnowModel_DALL_2006_01_01 {
dimensions:
	x = 160 ;
	y = 240 ;
variables:
	char crs ;
		crs:grid_mapping_name = "albers_conical_equal_area" ;
		crs:false_easting = 0. ;
		crs:false_northing = 0. ;
		crs:latitude_of_projection_origin = 50. ;
		crs:longitude_of_central_meridian = -154. ;
		crs:standard_parallel = 55., 65. ;
		crs:long_name = "CRS definition" ;
		crs:longitude_of_prime_meridian = 0. ;
		crs:semi_major_axis = 6378137. ;
		crs:inverse_flattening = 298.257222101004 ;
		crs:spatial_ref = "PROJCS[\"unnamed\",GEOGCS[\"NAD83\",DATUM[\"North_American_Datum_1983\",SPHEROID[\"GRS 1980\",6378137,298.2572221010042,AUTHORITY[\"EPSG\",\"7019\"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY[\"EPSG\",\"6269\"]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433],AUTHORITY[\"EPSG\",\"4269\"]],PROJECTION[\"Albers_Conic_Equal_Area\"],PARAMETER[\"standard_parallel_1\",55],PARAMETER[\"standard_parallel_2\",65],PARAMETER[\"latitude_of_center\",50],PARAMETER[\"longitude_of_center\",-154],PARA

### Example script to loop through 365 individual files for 2006; 1 file per day.

Windows:
```cmd
for %f in (*.nc) do (ncrename -h -v albers_conical_equal_area,crs %f)
```
UNIX:
```sh
for f in *.nc; do ncrename -h -v albers_conical_equal_area,crs $f; done
```

### Step 4.  Merge the 365 individual daily 2006 netCDF files into one netCDF file.  Add time dimension
---
We have generated 365 indiviuald daily netCDF files.  The next step is to merge the individual netCDFs into one file, where a record dimension is automatically created and named "time". 
This command excludes the "crs" variable so it will not be associated with the "time" dimension, but we will add the projection information back in in a subsequent step.


Use the NCO `ncecat` operator to merge all 2006 files
* [ncecat](https://linux.die.net/man/1/ncecat) – netCDF Ensemble conCATenator
> ncecat aggregates an arbitrary number of input files into a single output file
* `-x -v` : this combination of options will exclude the crs variable. It will not be written to the output file, but added back in a subsequent step.
* `-u time`: option names the new record dimension 'time'
* `-n ` : NCO decodes lists of such filenames encoded using the ‘-n’ syntax. Handy for some files. See documentation
>  -n takes the form -n file_number, digit_number, numeric_increment where
> *  file_number is the number of files, 
> * digit_number is the fixed number of numeric digits comprising the numeric_suffix, and 
> * numeric_increment is the constant, integer-valued difference between the numeric_suffix of any two consecutive files.
* A descriptive output filename is written **snow_depth_00500m_2006.nc**

In [9]:
# Step 4:  Merge netCDF files into one multidimensional file
ncecat -O -h -x -v crs -u time singleband/SnowModel_DALL_2006*.nc snow_depth_00500m_2006.nc

####  From the ncecat operation, the **time** dimension is added and properly attributed to the Band1 variable.  As noted above, the crs variable is not copied in the output file.

In [10]:
ncdump -h snow_depth_00500m_2006.nc

netcdf snow_depth_00500m_2006 {
dimensions:
	time = UNLIMITED ; // (365 currently)
	x = 160 ;
	y = 240 ;
variables:
	double x(x) ;
		x:standard_name = "projection_x_coordinate" ;
		x:long_name = "x coordinate of projection" ;
		x:units = "m" ;
	double y(y) ;
		y:standard_name = "projection_y_coordinate" ;
		y:long_name = "y coordinate of projection" ;
		y:units = "m" ;
	float lat(y, x) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
	float lon(y, x) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
	float Band1(time, y, x) ;
		Band1:long_name = "GDAL Band Number 1" ;
		Band1:_FillValue = -3.4e+38f ;
		Band1:grid_mapping = "albers_conical_equal_area" ;
		Band1:coordinates = "lon lat" ;

// global attributes:
		:GDAL_AREA_OR_POINT = "Area" ;
		:Conventions = "CF-1.5" ;
		:GDAL = "GDAL 2.3.3, released 2018/12/14" ;
		:history = "Mon Sep 02 19:45:39 2019: GDAL CreateCopy( SnowModel_DALL_2

### Step 5.  Copy the "crs" variable from one of the individual netCDFs to the merged one.

Use the NCO `ncks` to copy the `crs` variable from a previous netCDF daily file
* `ncks` – netCDF kitchen sink

In [11]:
#Step 5:  Copy the crs variable back into the merged file
ncks -h -A -v crs SnowModel_DALL_2006_01_01.nc snow_depth_00500m_2006.nc

In [12]:
ncdump -h snow_depth_00500m_2006.nc
# We'll show how to update the attribute: Band1:grid_mapping = "albers_conical_equal_area" ;

netcdf snow_depth_00500m_2006 {
dimensions:
	time = UNLIMITED ; // (365 currently)
	x = 160 ;
	y = 240 ;
variables:
	double x(x) ;
		x:standard_name = "projection_x_coordinate" ;
		x:long_name = "x coordinate of projection" ;
		x:units = "m" ;
	double y(y) ;
		y:standard_name = "projection_y_coordinate" ;
		y:long_name = "y coordinate of projection" ;
		y:units = "m" ;
	float lat(y, x) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
	float lon(y, x) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
	float Band1(time, y, x) ;
		Band1:long_name = "GDAL Band Number 1" ;
		Band1:_FillValue = -3.4e+38f ;
		Band1:grid_mapping = "albers_conical_equal_area" ;
		Band1:coordinates = "lon lat" ;
	char crs ;
		crs:grid_mapping_name = "albers_conical_equal_area" ;
		crs:false_easting = 0. ;
		crs:false_northing = 0. ;
		crs:latitude_of_projection_origin = 50. ;
		crs:longitude_of_central_meridian 

### Step 6.  Add the Time Variable and Time Values
* [ncap2](https://linux.die.net/man/1/ncap2) - netCDF Arithmetic Processor

In this case, use ncap2 to add time as a variable and populate the array with the 365 time values.
Recall that CF conventions for time:
> CF-compliant: as a numeric value with a defined start time, such as
>    "days since 2006-01-01 00:00:00"

Explain array

The ncdump "-t" option displays CF-compliant times in human readable form
```cmd
ncdump -t -v time snow_depth_00500m_2005.nc4
```

In [13]:
# !ncap2 -h -s 'time[$time]={0.5, 1.5, 2.5, 3.5, 4.5, ... , 364.5 };' snow_depth_00500m_2006.nc4 snow_depth_00500m_2006_time.nc4
ncap2 -O -h -s 'time=array(0.5, 1, $time);' snow_depth_00500m_2006.nc snow_depth_00500m_2006_time.nc

### Step 7.  Use `ncrename` to change the variable Band1 to something more descriptive

In [14]:
ncrename -h -v Band1,SnowDepth_500m snow_depth_00500m_2006_time.nc

### Step 8.  Use `ncatted` to update attributes to CF Compliance

An important part of netCDF is Standardizing files to CF Compliance.  Editing variables and attributes is often necessary. 

Use the NCO `ncatted` operator to edit variables and attributes
* [ncatted](https://linux.die.net/man/1/ncatted) – netCDF Attribute Editor
> ncatted edits attributes in a netCDF file. If you are editing attributes then you are spending too much time in the world of metadata, and ncatted was written to get you back out as quickly and painlessly as possible. ncatted can append, create, delete, modify, and overwrite attributes.

See the link associated with ncatted.  It's very powerful once understood.

In [15]:
## update attributes for the time variable
ncatted -h -a units,time,c,c,"days since 2006-01-01 00:00:00" snow_depth_00500m_2006_time.nc
ncatted -h -a calendar,time,c,c,"standard" snow_depth_00500m_2006_time.nc
ncatted -h -a description,time,c,c,"middle of each day" snow_depth_00500m_2006_time.nc
ncatted -h -a long_name,time,c,c,"time" snow_depth_00500m_2006_time.nc
ncatted -h -a standard_name,time,c,c,"time" snow_depth_00500m_2006_time.nc

In [16]:
# The ncdump "-t" option displays CF-compliant times in human readable form
ncdump -t -v time snow_depth_00500m_2006_time.nc

netcdf snow_depth_00500m_2006_time {
dimensions:
	time = UNLIMITED ; // (365 currently)
	x = 160 ;
	y = 240 ;
variables:
	double time(time) ;
		time:units = "days since 2006-01-01 00:00:00" ;
		time:calendar = "standard" ;
		time:description = "middle of each day" ;
		time:long_name = "time" ;
		time:standard_name = "time" ;
	double x(x) ;
		x:standard_name = "projection_x_coordinate" ;
		x:long_name = "x coordinate of projection" ;
		x:units = "m" ;
	double y(y) ;
		y:standard_name = "projection_y_coordinate" ;
		y:long_name = "y coordinate of projection" ;
		y:units = "m" ;
	float lat(y, x) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
	float lon(y, x) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
	float SnowDepth_500m(time, y, x) ;
		SnowDepth_500m:long_name = "GDAL Band Number 1" ;
		SnowDepth_500m:_FillValue = -3.4e+38f ;
		SnowDepth_500m:grid_mapping = "albers_conical_equa

    "2006-11-17 12", "2006-11-18 12", "2006-11-19 12", "2006-11-20 12", 
    "2006-11-21 12", "2006-11-22 12", "2006-11-23 12", "2006-11-24 12", 
    "2006-11-25 12", "2006-11-26 12", "2006-11-27 12", "2006-11-28 12", 
    "2006-11-29 12", "2006-11-30 12", "2006-12-01 12", "2006-12-02 12", 
    "2006-12-03 12", "2006-12-04 12", "2006-12-05 12", "2006-12-06 12", 
    "2006-12-07 12", "2006-12-08 12", "2006-12-09 12", "2006-12-10 12", 
    "2006-12-11 12", "2006-12-12 12", "2006-12-13 12", "2006-12-14 12", 
    "2006-12-15 12", "2006-12-16 12", "2006-12-17 12", "2006-12-18 12", 
    "2006-12-19 12", "2006-12-20 12", "2006-12-21 12", "2006-12-22 12", 
    "2006-12-23 12", "2006-12-24 12", "2006-12-25 12", "2006-12-26 12", 
    "2006-12-27 12", "2006-12-28 12", "2006-12-29 12", "2006-12-30 12", 
    "2006-12-31 12" ;
}


#### A little more editing on variable attributes to get to CF Compliance
---

In [17]:
## update attributes for the SnowDepth_500m variable
ncatted -h -a grid_mapping,SnowDepth_500m,o,c,"crs" snow_depth_00500m_2006_time.nc
ncatted -h -a long_name,SnowDepth_500m,o,c,"SnowModel snow depth: 500-m resolution" snow_depth_00500m_2006_time.nc
ncatted -h -a units,SnowDepth_500m,c,c,"cm" snow_depth_00500m_2006_time.nc

## update attributes in the Global section
ncatted -h -a Conventions,global,o,c,"CF-1.6" snow_depth_00500m_2006_time.nc
ncatted -h -a institution,global,c,c,"University of Washington" snow_depth_00500m_2006_time.nc
ncatted -h -a project,global,c,c,"Arctic-Boreal Vulnerability Experiment (ABoVE)" snow_depth_00500m_2006_time.nc
ncatted -h -a references,global,c,c,"Mahoney, P., Liston, G., Mangipane, B., Prugh, L." snow_depth_00500m_2006_time.nc

## Congratulations!  You've created a CF Compliant netCDF File
---

In [18]:
# Examine the header of the complete file
ncdump -h snow_depth_00500m_2006_time.nc

netcdf snow_depth_00500m_2006_time {
dimensions:
	time = UNLIMITED ; // (365 currently)
	x = 160 ;
	y = 240 ;
variables:
	double time(time) ;
		time:units = "days since 2006-01-01 00:00:00" ;
		time:calendar = "standard" ;
		time:description = "middle of each day" ;
		time:long_name = "time" ;
		time:standard_name = "time" ;
	double x(x) ;
		x:standard_name = "projection_x_coordinate" ;
		x:long_name = "x coordinate of projection" ;
		x:units = "m" ;
	double y(y) ;
		y:standard_name = "projection_y_coordinate" ;
		y:long_name = "y coordinate of projection" ;
		y:units = "m" ;
	float lat(y, x) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
	float lon(y, x) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
	float SnowDepth_500m(time, y, x) ;
		SnowDepth_500m:long_name = "SnowModel snow depth: 500-m resolution" ;
		SnowDepth_500m:_FillValue = -3.4e+38f ;
		SnowDepth_500m:grid_mapping = 

---
## Put all the steps together into a simple script file
---

### There were a lot of steps in the demonstration, but it's not necessary to examine each output file as in the demonstration above.  After the steps are worked out, put it all together into a final script file that can be easily executed, edited, and repeated as needed.

################################################################################################

This is a sample set of commands that can be easily scripted to transform a series of indiviual 
geoTIFF files into a multidimensional netCDF file.  

################################################################################################

~~~sh
####
# Sample command line fucntions to demonstrate converted a series of "daily" geoTIFF files into a multidimensional netCDF file
# Assumes the script is started in the directory containing sample geoTIFF files from the 
####

# Make sure the band1 subdirectory exists
if [-d //band1]; then
    echo //band1
else
    mkdir //band1
fi

# Step 1: Extract SnowDepth band from geoTIFF files
for f in *.tif; do gdal_translate -b 1 $f band1/$f; done

cd band1
# Step 2: Convert geoTIFF to netCDF
for f in *.tif; do gdal_translate -of netCDF -co WRITE_LONLAT=YES $f ${f%.*}.nc; done

# Step 3: Rename albers_conical_equal_area variable to crs
for f in *.nc; do ncrename -h -v albers_conical_equal_area,crs $f; done

# Step 4:  Merge netCDF files into one multidimensional file
ncecat -h -x -v crs -u time SnowModel_DALL_2006*.nc snow_depth_00500m_2006.nc

# Step 5:  Copy the crs variable back into the merged file
ncks -h -A -v crs SnowModel_DALL_2006_01_01.nc snow_depth_00500m_2006.nc

# Step 6:  Add the time variable and values
ncap2 -h -s 'time=array(0.5, 1, $time);' snow_depth_00500m_2006.nc snow_depth_00500m_2006_time.nc

# Step 7: Rename the Band1 variable to SnowDepth_500m
ncrename -h -v Band1,SnowDepth_500m snow_depth_00500m_2006_time.nc

# Step 8:  Update attributes to CF Compliance
## update attributes for the time variable
ncatted -h -a units,time,c,c,"days since 2006-01-01 00:00:00" snow_depth_00500m_2006_time.nc
ncatted -h -a calendar,time,c,c,"standard" snow_depth_00500m_2006_time.nc
ncatted -h -a description,time,c,c,"middle of each day" snow_depth_00500m_2006_time.nc
ncatted -h -a long_name,time,c,c,"time" snow_depth_00500m_2006_time.nc
ncatted -h -a standard_name,time,c,c,"time" snow_depth_00500m_2006_time.nc

## update attributes for the SnowDepth_500m variable
ncatted -h -a grid_mapping,SnowDepth_500m,o,c,"crs" snow_depth_00500m_2006_time.nc
ncatted -h -a long_name,SnowDepth_500m,o,c,"SnowModel snow depth: 500-m resolution" snow_depth_00500m_2006_time.nc
ncatted -h -a units,SnowDepth_500m,c,c,"cm" snow_depth_00500m_2006_time.nc

## update attributes in the Global section
ncatted -h -a Conventions,global,o,c,"CF-1.6" snow_depth_00500m_2006_time.nc
ncatted -h -a institution,global,c,c,"University of Washington" snow_depth_00500m_2006_time.nc
ncatted -h -a project,global,c,c,"Arctic-Boreal Vulnerability Experiment (ABoVE)" snow_depth_00500m_2006_time.nc
ncatted -h -a references,global,c,c,"Mahoney, P., Liston, G., Mangipane, B., Prugh, L." snow_depth_00500m_2006_time.nc
~~~